In [1]:
# I used this notebook to run scGen on all the 4i data for all drugs

# This follows https://scgen.readthedocs.io/en/stable/tutorials/scgen_perturbation_prediction.html

In [2]:
# I had to change something in scvi/model/base/_vaemixin.px since the scgen outputs where not read proberly

# line 304

#                if str(self.__class__) == "<class 'scgen._scgen.SCGEN'>":

#                    qzm: Tensor = outputs.get("qz_m")
#                    qzv: Tensor = outputs.get("qz_v")
#                    qz: Distribution = Normal(qzm, qzv.sqrt())
                
#               	else: 

#                    qzm: Tensor = outputs.get(MODULE_KEYS.QZM_KEY)
#                    qzv: Tensor = outputs.get(MODULE_KEYS.QZV_KEY)
#                    qz: Distribution = Normal(qzm, qzv.sqrt())

In [3]:
pip freeze --local

absl-py==2.1.0
adjustText==1.3.0
aiohappyeyeballs==2.4.6
aiohttp==3.11.12
aiosignal==1.3.2
anndata==0.11.0
array_api_compat @ file:///home/conda/feedstock_root/build_artifacts/array-api-compat_1741126920022/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
attrs==25.1.0
cached-property @ file:///home/conda/feedstock_root/build_artifacts/cached_property_1615209429212/work
chex==0.1.88
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1733218098505/work
comm @ file:///croot/comm_1709322850197/work
contourpy @ file:///home/conda/feedstock_root/build_artifacts/contourpy_1731428301407/work
cycler @ file:///home/conda/feedstock_root/build_artifacts/cycler_1733332471406/work
debugpy @ file:///croot/debugpy_1736267418885/work
decorator @ file:///opt/conda/conda-bld/decorator_1643638310831/work
dill @ file:///home/conda/feedstock_root/build_artifacts/dill_1733249551891/work
docrep==0.3.2
doubletdetection==4.3.0.post1
etils==1.12.0
exceptiongroup @ fil

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import scanpy as sc
import numpy as np
import scgen
import anndata
from tqdm import tqdm
import scvi

/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('/home/manu/cellot-main/datasets/4i/drugs.txt', header=None)
drugs = list(df[0])
drugs = [a[:-1] if a!='vindesine' else a for a in drugs]

In [6]:
for drug in drugs:
    print(drug)
    adata=sc.read_h5ad('/home/manu/cellot-main/datasets/4i/8h.h5ad')
    adata.obs['cell_type'] = '4i'
    features = pd.read_csv('/home/manu/cellot-main/datasets/4i/features.txt', header=None)
    features = list(features[0])
    adata = adata[:, features].copy()
    adata = adata[adata.obs.drug.isin(['control', drug])].copy()
    df = pd.read_csv(f'/home/manu/Desktop/cellflow/data/splits/{drug}.csv')
    train_cells = list(df[df['split']=='train']['Unnamed: 0'])
    test_cells = list(df[df['split']=='test']['Unnamed: 0'])

    adata_train = adata[train_cells].copy()
    adata_test = adata[test_cells].copy()

    scgen.SCGEN.setup_anndata(adata_train, batch_key='drug', labels_key="cell_type")
    scvi.model.SCVI.setup_anndata(adata_train, batch_key="drug", labels_key="cell_type")
    scgen.SCGEN.setup_anndata(adata_train, batch_key='drug', labels_key="cell_type")
    model = scgen.SCGEN(adata_train)
    model.train(max_epochs=100, batch_size=128, early_stopping=True, early_stopping_patience=25)
    pred, delta = model.predict(ctrl_key='control', stim_key=drug, celltype_to_predict='4i')
    pred.obs['split']='test'
    pred.write(f'/home/manu/Desktop/cellflow/data/scgen_preds/imputed_{drug}_single.h5ad')

cisplatin


/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in th

Epoch 26/100:  26%|▎| 26/100 [00:19<00:55,  1.33it/s, v_num=1, train_loss_step=0
Monitored metric elbo_validation did not improve in the last 25 records. Best score: 48.151. Signaling Trainer to stop.
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


cisplatin_olaparib


...

/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


vindesine


/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in th

Epoch 27/100:  27%|▎| 27/100 [00:19<00:53,  1.36it/s, v_num=1, train_loss_step=0
Monitored metric elbo_validation did not improve in the last 25 records. Best score: 46.257. Signaling Trainer to stop.
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/manu/miniconda3/envs/scvi_env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
